In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
DATA_MEAN = 168.3172158554484
DATA_STD = 340.21625683608994
OUTPUT_CHANNELS = 1
DATA_PATH = "/home/matthew/Documents/unet_model/6375_images/"
WEIGHTS = "/home/matthew/Documents/unet_model/dsc_after_bce_old.hdf5"

In [ ]:
import unet as unet

model = unet.model(output_channels=OUTPUT_CHANNELS)
model.compile()
model.load_weights(WEIGHTS)

In [ ]:
import glob

dcm_paths = glob.glob(DATA_PATH + "/*.dcm")

In [ ]:
import pydicom

dcms = [pydicom.dcmread(path, force=True) for path in dcm_paths]

In [ ]:
sorted_dcms = sorted(dcms, key=lambda dcm: dcm.SliceLocation)

In [ ]:
def get_image_instance(dcm):
    try:
        dcm.file_meta.TransferSyntaxUID
    except AttributeError:
         dcm.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
    return dcm.pixel_array

In [ ]:
import numpy as np

imgs = np.array([get_image_instance(dcm) for dcm in sorted_dcms])
imgs = imgs[..., np.newaxis]
imgs = (imgs - DATA_MEAN) / DATA_STD

In [ ]:
predicts = model.predict(x=imgs, batch_size=1, verbose=1)
predicts = np.round(predicts)

In [ ]:
from pymedphys._experimental.autosegmentation import mask
from matplotlib import pyplot as plt

In [ ]:
index = 1
dcm = sorted_dcms[index]
pred = predicts[index]

z = dcm.SliceLocation

x_grid, y_grid, ct_size = mask.get_grid(dcm)
slice_contours = mask.get_contours_from_mask(x_grid, y_grid, pred[...,0])

slice_contours_xyz = []
for roi in slice_contours:
    roi_xyz = []
    for xy_point in roi:
        xyz_point = [*xy_point, z]
        roi_xyz = roi_xyz + xyz_point
    slice_contours_xyz.append(roi_xyz)

In [ ]:
for roi in slice_contours_xyz: 
    print(roi)
    print("-------------")